In [78]:
from sklearn import model_selection, datasets, svm
from qiskit import QuantumCircuit, Aer, IBMQ, QuantumRegister, ClassicalRegister
import qiskit
import numpy as np
import copy

In [45]:
iris=datasets.load_iris()#binaryy-clssification using quatum svm
X=iris.data[0:100]
Y=iris.target[0:100]
X_train,X_test,Y_train,Y_test=model_selection.train_test_split(X,Y,test_size=0.33,random_state=42)

In [94]:
print(Y_train[0])#classifying between two classes of flowers
print(X_train[0])
N=4#number of features

0
[5.2 3.4 1.4 0.2]


In [95]:
q=QuantumRegister(N)#number of qubits in quantum-circuit== number of features
c= ClassicalRegister(1)#single register which will have output

qc=QuantumCircuit(q,c)
qc.draw()

q197344_0: 
           
q197344_1: 
           
q197344_2: 
           
q197344_3: 
           
 c15196: 1/

In [96]:
#for i,x in enumerate(X_train[0]):
  #  qc.rx(x,i)#rotate qubit by 4 features of a sinle flower(datapoint)
#qc.draw()####feature data mapped onto qubit state


In [97]:
def feature_map(X):
    q=QuantumRegister(N)
    c=ClassicalRegister(1)
    qc=QuantumCircuit(q,c)
    for i,x in enumerate(X):
        qc.rx(x,i)
    return (qc,c)

In [98]:
qc,c=feature_map(X_train[3])
qc.draw()

┌─────────┐
q197345_0: ┤ Rx(5.6) ├
           └┬───────┬┘
q197345_1: ─┤ Rx(3) ├─
           ┌┴───────┴┐
q197345_2: ┤ Rx(4.5) ├
           ├─────────┤
q197345_3: ┤ Rx(1.5) ├
           └─────────┘
 c15197: 1/═══════════

In [99]:
#now we need to measure 
qc.measure(0,c)#measure 0 qubit and store in clasical register c
qc.draw()

┌─────────┐┌─┐
q197345_0: ┤ Rx(5.6) ├┤M├
           └┬───────┬┘└╥┘
q197345_1: ─┤ Rx(3) ├──╫─
           ┌┴───────┴┐ ║ 
q197345_2: ┤ Rx(4.5) ├─╫─
           ├─────────┤ ║ 
q197345_3: ┤ Rx(1.5) ├─╫─
           └─────────┘ ║ 
 c15197: 1/════════════╩═
                       0

In [100]:
def variational_circuit(qc,theta):
    for i in range(N-1):
        qc.cnot(i,i+1)
    qc.cnot(N-1,0)
    for i in range(N):
        qc.ry(theta[i],i)
    return qc


In [101]:
qc,c=feature_map(X_train[3])
qc.barrier()

qc=variational_circuit(qc,np.random.rand(N))

qc.barrier()

qc.measure(0,c)
qc.draw()####idea is to lean the rotation-angles in variational_ircuit ie theta such that q_ircuit performs

┌─────────┐ ░                          ┌───┐┌──────────────┐ ░ ┌─┐
q197346_0: ┤ Rx(5.6) ├─░───■──────────────────────┤ X ├┤ Ry(0.065557) ├─░─┤M├
           └┬───────┬┘ ░ ┌─┴─┐     ┌─────────────┐└─┬─┘└──────────────┘ ░ └╥┘
q197346_1: ─┤ Rx(3) ├──░─┤ X ├──■──┤ Ry(0.99278) ├──┼───────────────────░──╫─
           ┌┴───────┴┐ ░ └───┘┌─┴─┐└─────────────┘  │  ┌─────────────┐  ░  ║ 
q197346_2: ┤ Rx(4.5) ├─░──────┤ X ├───────■─────────┼──┤ Ry(0.85833) ├──░──╫─
           ├─────────┤ ░      └───┘     ┌─┴─┐       │  ├─────────────┤  ░  ║ 
q197346_3: ┤ Rx(1.5) ├─░────────────────┤ X ├───────■──┤ Ry(0.80351) ├──░──╫─
           └─────────┘ ░                └───┘          └─────────────┘  ░  ║ 
 c15198: 1/════════════════════════════════════════════════════════════════╩═
                                                                           0

In [102]:
shots=1E4
backend=Aer.get_backend('qasm_simulator')
job=qiskit.execute(qc,backend,shots=shots)
result=job.result()
counts=result.get_counts(qc)
print(counts)
print(counts['1']/shots)#result of circuit 

{'0': 5036, '1': 4964}
0.4964


In [103]:
def quantum_nn(X,theta,simulator=True):
    qc,c=feature_map(X)
    qc.barrier()
    qc=variational_circuit(qc,theta)
    qc.barrier()
    qc.measure(0,c)
    shots=1E4
    backend=Aer.get_backend('qasm_simulator')
    
    if simulator==False:
        shots=5000
        provider=IBMQ.load_account()
        nackend=provider.get_backend('')

    
    job=qiskit.execute(qc,backend,shots=shots)
    result=job.result()
    counts=result.get_counts(qc)
    return counts['1']/shots

In [104]:
def loss(prediction,target):
    return (prediction-target)**2

In [105]:
prediction=quantum_nn(X_train[6],np.random.rand(N))
target=Y_train[6]
###minimise loss to train theta

In [92]:
def gradient(X,Y,theta):
    delta=0.01
    grad=[]
    for i in range(len(theta)):
        dtheta=copy.copy(theta)
        dtheta[i]+=delta
        pred1=quantum_nn(X,dtheta)
        pred2=quantum_nn(X,theta)
        
        grad.append((loss(pred1,Y)-loss(pred2,Y))/delta)
    return np.array(grad)

def accuracy(X,Y,theta):
    counter=0
    for X_i,Y_i in zip(X,Y):
        predict=quantum_nn(X_i,theta)
        if predict<0.5 and Y_i==0:
            counter+=1
        elif predict>=0.5 and Y_i==1:
            counter+=1
    return counter/len(Y)


eta=0.05
theta=np.random.rand(N)
losslist=[]

for i in range(10):
    losstmp=[]
    for X_i,Y_i in zip(X_train,Y_train):
        predict=quantum_nn(X_i,theta)
        losstmp.append(loss(predict,Y_i))
        #update theta using stochastic gradient descent
        theta=theta-eta*gradient(X_i,Y_i,theta)
    losslist.append(np.mean(losstmp))
    acc=accuracy(X_train,Y_train,theta)
    print(f'{i} \ t {losslist[-1] :.3f} \t {acc:.3f}')

0 \ t 0.292 	 0.134
1 \ t 0.292 	 0.164
2 \ t 0.296 	 0.194
3 \ t 0.282 	 0.179
4 \ t 0.278 	 0.164
5 \ t 0.256 	 0.821
6 \ t 0.227 	 0.791
7 \ t 0.227 	 0.776
8 \ t 0.230 	 0.791
9 \ t 0.209 	 0.851


In [93]:
accuracy(X_test,Y_test,theta)

0.7878787878787878